<a href="https://colab.research.google.com/github/rssubramaniyan1/EVA8/blob/main/EVA8_Assignment6_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
import torch
import cv2
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F
import torch.nn as nn

In [ ]:
#function to calculate mean and standard deviation of the dataset

def calculate_mean_std(dataset):
    # shape (50000, 32, 32, 3)
    imgs = np.concatenate(dataset, axis=0)/255.0
    # the red channel across all images
    r = imgs[:, :, 0].mean()
    # the green channel across all images
    g = imgs[:, :, 1].mean()
    # the blue channel across all images
    b = imgs[:, :, 2].mean()
    # the red channel across all images
    r_std = imgs[:, :, 0].std()
    # the green channel across all images
    g_std = imgs[:, :, 1].std()
    # the blue channel across all images
    b_std = imgs[:, :, 2].std()
    return [r, g, b], [r_std, g_std, b_std]

In [ ]:
# apply transformations on the train dataset

def apply_train_transforms(train_set, mean_train, std_train, batch_size, kwargs):
    train_transforms = A.Compose([
            A.HorizontalFlip(p=0.5),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
            A.CoarseDropout(max_holes=1, max_height=16, max_width=16, min_holes=1, min_height=16, min_width=16,
                        fill_value=(mean_train), p=0.5),
            A.Normalize(mean_train, std_train),
            ToTensorV2()
            ])

    # Applying the transformations on the train dataset and creating a train loader
    # reshape to HWC format
    train_dataset = train_set.data.transpose((0, 1, 2, 3))
    train_labels = train_set.targets
    # get the train_dataset in the required format for albumentations library to work cv2 format
    #train_dataset = np.array([cv2.cvtColor(train_dataset[p], cv2.COLOR_BGR2RGB) for p in range(train_dataset.shape[0])])
    train_dataset=[train_transforms(image=train_dataset[p]) for p in range(train_dataset.shape[0])]
    # Convert the list of dictionaries to a torch.Tensor
    train_dataset = torch.stack([x['image'] for x in train_dataset])
    # Transpose the tensor to the required shape (N, C, H, W) i.e. (50000, 3, 32, 32)
    train_dataset = train_dataset.transpose(2, 1).transpose(1,2)
    train_dataset = torch.utils.data.TensorDataset(train_dataset, torch.tensor(train_labels))
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
    return train_loader


In [ ]:

def apply_test_transforms(test_set, mean_test, std_test, batch_size, kwargs):
    test_transforms = A.Compose([
            A.Normalize(mean_test, std_test),
            ToTensorV2()
            ])
    # Applying the transformations on the test dataset and creating a test loader
    test_dataset = test_set.data.transpose((0, 1, 2, 3))
    test_labels = test_set.targets
    test_dataset = [test_transforms(image = test_dataset[p]) for p in range(test_dataset.shape[0])]
    # Convert the list of dictionaries to a torch.Tensor
    test_dataset = torch.stack([x['image'] for x in test_dataset])
    # transpose not required as the test dataset is already in the required shape (N, C, H, W) i.e. (10000, 3, 32, 32)
    test_dataset = torch.utils.data.TensorDataset(test_dataset, torch.tensor(test_labels))
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, **kwargs)
    return test_loader


In [ ]:

# write a function for the optimizer and scheduler

def get_optimizer(model,learning_rate,momentum,weight_decay,step_size,gamma):# Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
    scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
    return criterion, optimizer, scheduler


In [ ]:

train_losses = []
train_acc = []
test_losses = []
test_acc = []



In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        # get samples
        data, target = data.to(device), target.to(device)

        # Init
        optimizer.zero_grad()
        # Predict
        y_pred = model(data)

        # Calculate loss
        loss = F.nll_loss(y_pred, target)
        train_losses.append(loss)

        # Backpropagation
        loss.backward()
        optimizer.step()

        # Update pbar-tqdm
        pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        pbar.set_description(desc= f'Loss={loss.item():0.2f} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
        train_acc.append(100*correct/processed)

In [ ]:
# Test the model
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
